# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configuration`. Nel dettaglio la funzione deve:
    * Dividere il dataset in tr e vidation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di vidation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di vidation 

    * Il migliore MAE di tr

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [7]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [8]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configuration = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non va lide
for config in configuration:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configuration = set([tuple(config.items()) for config in configuration])

# Convertiamo di nuovo in lista di dizionari
configuration = [dict(config) for config in configuration]
print(f'Numero di combinazioni: {len(configuration)}')

Numero di combinazioni: 56


In [12]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_tr, y_tr, X_v, y_v, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
   if hyperparams['use_pca']:
        if hyperparams['pca_standardize']:
            scaler_pca = StandardScaler().fit(X_tr)
            X_tr = scaler_pca.transform(X_tr)
            X_v = scaler_pca.transform(X_v)
        pca = PCA(n_components=hyperparams['pca_components'])
        X_tr = pca.fit_transform(X_tr)
        X_v = pca.transform(X_v)

    if hyperparams['data_standardize']:
        scaler = StandardScaler().fit(X_tr)
        X_tr = scaler.transform(X_tr)
        X_v = scaler.transform(X_v)



    # Aggiunge il termine costante ai dati
    # ...

    X_tr = np.c_[np.ones(len(X_tr)), X_tr]
    X_v  = np.c_[np.ones(len(X_v)),  X_v]
    

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        I = np.eye(X_tr.shape[1])
        I[0,0] = 0
        parametri = (np.linalg.inv(X_tr.T @ X_tr + hyperparams['reg_lambda'] * I)) @ X_tr.T @ y_tr
    else:
        parametri = (np.linalg.inv(X_tr.T @ X_tr)) @ X_tr.T @ y_tr
        

    # Calcolo predizioni
    # ...

    y_tr_pred = X_tr @ parametri
    y_v_pred = X_v @ parametri
    # Calcola il MAE
    # ...

    MAE_tr = np.mean(np.abs(y_tr - y_tr_pred))
    MAE_v = np.mean(np.abs(y_v - y_v_pred))

    return MAE_tr, MAE_v, parametri

# Dividi il dataset in tring e test set
# ...

porzione_tr = 0.8
porzione_test = 0.2
num_tr = int(porzione_tr * X.shape[0])
X_tr = X[:num_tr]
y_tr = y[:num_tr]
X_test = X[num_tr:]
y_test = y[num_tr:]

# Dividi il training set in training set effettivo e validation set
# ...

porzione_tr_ef = 0.8
porzione_v = 0.2
num_tr_ef = int(porzione_tr_ef * X_tr.shape[0])
X_tr_ef = X_tr[:num_tr_ef]
y_tr_ef = y_tr[:num_tr_ef]
X_v = X_tr[num_tr_ef:]
y_v = y_tr[num_tr_ef:]
# Trova la configurazione di iperparametri migliore
# ...

improve_config = None
improve_mae_v = float('inf')
improve_mae_tr = None
   
for config in configuration:
       # X_tr_ef = X_tr_ef.copy()
       #X_v = X_v.copy() 
    mae_tr, mae_v, _ = pipeline(X_tr_ef, y_tr_ef, X_v, y_v, config)
        
    if mae_v < improve_mae_v:
            
        improve_mae_v = mae_v
        improve_config = config
        improve_mae_tr = mae_tr

# Riallena il modello sul tring set completo
# ...

_, _, improve_parametri = pipeline(X_tr, y_tr, X_test, y_test, improve_config)

# Calcola il MAE sul test set
# ...

X_test_copy = X_test.copy()
if improve_config['use_pca']:
    if improve_config['pca_standardize']:
        X_tr = scaler_pca.fit_transform(X_tr)
        X_test_copy = scaler_pca.transform(X_test_copy)
    pca = PCA(n_components=improve_config['pca_components'])
    X_tr = pca.fit_transform(X_tr)
    X_test_copy = pca.transform(X_test_copy)
    

if improve_config['data_standardize']:
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_tr)
    X_test_copy = scaler.transform(X_test_copy)

X_tr = np.c_[np.ones((X_tr.shape[0], 1)), X_tr]
X_test_copy = np.c_[np.ones((X_test_copy.shape[0], 1)), X_test_copy]

y_test_pred = X_test_copy @ improve_parametri
mae_test = np.mean(np.abs(y_test - y_test_pred))

# Stampa  risultati
# ...

print(f"La miglior combinazione degli iperparametri è: \n{improve_config}")
print(f"Il risultato migliore del MAE calcolato sul validation set è: {improve_mae_v:.4f}")
print(f"Il MAE calcolato sul tring set con i migliori parametri è: {improve_mae_tr:.4f}")
print(f"Il MAE calcolato sul test set con i migliori parametri è: {mae_test:.4f}")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

In `configuration` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di tring. 